In [89]:
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.special import gamma
import time

In [90]:
# Inicia o cronômetro
start_time = time.time()

n = 5*10**6 # Número de vetores thetas
k = 17*10**3 # Número de bins
m = 6 # Dimensionalidade
x = np.random.randint(0, 11, m) # Vetor x
y = np.random.randint(0, 11, m) # Vetor y

alfa = x + y # Vetor Alfa = Vetor x + Vetor y
alfa

array([12, 17, 16, 16, 14, 11])

In [91]:
def f(theta, alfa, constante):
    log_produtorio = np.sum((alfa - 1) * np.log(theta), axis=1) # Para acelerar as operações, faz-se log do produtório (vira somatório)
    return np.exp(log_produtorio)/constante

def beta_multivariavel(alfa): # Beta(a + b) = Gamma(a)Gamma(b)/Gamma(a + b)
    log_produtorio = np.sum(np.log(gamma(alfa)))
    return np.exp(log_produtorio)/gamma(np.sum(alfa))

In [92]:
def gerador_dirichlet(alfa, n): # Essa função gera amostras dirichlet utilizando a função gamma: y_i = x_i/sum(x)
    # Gera amostras da distribuição gamma
    samples = np.random.gamma(alfa, 1, size=(n, m))
    
    # Normalize as amostras para que a soma de cada vetor seja 1
    thetas = samples / np.sum(samples, axis=1)[:, np.newaxis]

    return thetas

In [93]:
def criar_lista_v(lista_f_ord, n, k):
    # Define-se os pontos de corte v
    # - v_0 = 0; v_k = sup(f(theta))
    resto = n % k
    tamanho_bin = n // k

    # No entanto, quando n%k = c != 0, precisa-se adaptar o tamanho de c bins com um ponto a mais
    ajuste = np.concatenate((np.arange(1, resto + 1), np.array([resto] * (k - resto)))) # Array para ajustar o tamanho dos bins
    lista_intermed = np.arange(1, k + 1) * tamanho_bin + ajuste
    lista_v = np.concatenate(([0], lista_f_ord[lista_intermed - 1]))
    return lista_v, lista_intermed

In [94]:
thetas = gerador_dirichlet(alfa, n) # Lista de Vetores theta

constante = beta_multivariavel(alfa)
lista_f = f(thetas, alfa, constante) # Aplicar f em cada vetor theta

In [95]:
lista_f_ord = np.sort(lista_f) # Ordena a lista de resultados
sup_f = lista_f_ord[-1] # Define o supremo de f(theta) como o valor máximo obtido

lista_v, lista_intermed = criar_lista_v(lista_f_ord, n, k)

In [96]:
from scipy.interpolate import PchipInterpolator

In [97]:
fracao = np.concatenate((np.array([0]), lista_intermed))/n # Identifica a fração total de pontos de 0 até o ponto v

interp = PchipInterpolator(lista_v, fracao)
# Utiliza um interpolador PCHIP para gerar um polinômio de terceiro grau que aproxime W(v)
# Dessa forma, obtém-se U(v)

# Define os pontos onde deseja-se avaliar a função interpolada
x_new = np.linspace(0, lista_v[-1], 100)

# Avalia a função interpolada nos novos pontos
y_new = interp(x_new)

In [98]:
# Termina o cronômetro
end_time = time.time()

# Tempo decorrido em segundos
elapsed_time = end_time - start_time
print(f"Tempo decorrido: {elapsed_time} segundos")

Tempo decorrido: 2.2138092517852783 segundos


In [99]:
z_score = 1.96
erro_n = z_score/(2*(n**(1/2)))
erro_k = 1/k
print(erro_n, erro_k)

erro_total = erro_n + erro_k
erro_total*(10**2)

0.00043826932358995873 5.882352941176471e-05


0.04970928530017235